In [127]:
import random
from typing import List

class Game():
    def __init__(self, players: int):
        """
        Initialize a trio game with a specified number of players.
        
        Parameters:
        players (int): The number of players participating in the game.
        """
        # Initialize deck
        self.deck = [x for x in range(1,13)] * 3
        random.shuffle(self.deck)
        self.trios = [] # List of trios found

        # Constants (rules)
        CARDS_PER_PLAYER_DICT = {3: 9, 4: 7, 5: 6, 6: 5}
        CARDS_PER_PLAYER = CARDS_PER_PLAYER_DICT[players]

        # Initialize players, table and results
        print("Distributing cards...")
        self.players = [Player("Player " + str(x), self.deck[x*CARDS_PER_PLAYER:(x+1)*CARDS_PER_PLAYER], self) for x in range(players)]
        self.table = Table(self.deck[players*CARDS_PER_PLAYER:], self)


    def end_turn(self):
        """
        End the current turn and start the next one.
        """
        for p in self.players:
            p.hide_all()
        self.table.hide_all()

    def knowledge(self):
        for p in self.players:
            print(f"{p.name}:")
            print([p.hand[i] if p.shown[i] else "?" for i in range(len(p.hand))])
        print("Table:")
        print([self.table.cards[i] if self.table.shown[i] else "?" for i in range(len(self.table.cards))])

    def trio_found(self, player, card: int):
        print(f"Player {player.name} found a trio of {card}!")

        # Remove the trio from the table and the player's hand
        while card in self.table.cards:
            self.table.cards.remove(card)
        for p in self.players:
            while card in p.hand:
                p.hand.remove(card)
        
        # Add trio to the player's collection
        player.trios.append(card)
        player.score += 1
        self.trios.append((player, card)) # This is useful for strategy
        self.check_victory(player)
    
    def check_victory(self, player):
        if player.score == 3:
            print(f"Player {player.name} has reached 3 trios and won the game!")
            return True
        elif 7 in player.trios:
            print(f"Player {player.name} has found a trio of 7 and won the game!")
            return True
        else:
            return False

    def print_game(self):
        print(str(self.table))
        for p in self.players:
            print(str(p))

    def __str__(self):
        return str(self.cards)


class Hand():
    def __init__(self, cards: List[int]):
        self.cards = cards
        # Now we need to keep track of:
        # - Which cards are visible in a turn
        self.visible = [False for x in range(len(self.cards))]
        # - Which cards have been shown (current knowledge of the game)
        self.shown = [False for x in range(len(self.cards))]
    
    def show(self, index):
        """
        Update visibility status of a card if it is shown
        """
        self.visible[index] = True
        self.shown[index] = True
        return self.cards[index]
    
    def hide_all(self):
        self.visible = [False for _ in self.visible]
    
    def all_shown(self):
        return sum(self.shown) == len(self.cards)


class Player(Hand):
    def __init__(self, name: str, hand: List[int], game: Game):
        self.game = game  # Reference to the game object
        self.name = name  # Name of the player
        self.trios = []  # Collection of trios found
        self.score = 0  # Score of the player
        super().__init__(hand) # Initialize the cards with their visibility and shown status
        self.cards.sort() # Sorted list of cards in the player's hand
        self.hand = self.cards  # Just to have the other variable name

        # Check if there are any trios in the hand
        self.discard_trios()

    def __str__(self):
        return self.name + ":\n" + str(self.hand)
    
    def discard_trios(self):
        """
        Find a standing trio in the hand and discard it
        """
        checked = []
        for i in self.hand:
            if i not in checked:
                if self.hand.count(i) == 3:
                    print(f"{self.name} discarded trio {i}!")
                    self.trios.append(i)
                    self.score += 1
                    self.game.trios.append(i)
                    for j in range(3):
                        self.hand.remove(i)
                    return True
                checked.append(i)

    def get_other_players(self):
        """
        Get a list of all other players in the game
        """
        return [p for p in self.game.players if p != self]
    
    def get_random_player(self):
        """
        Get a random player from the game
        """
        # TODO: At some point, we should check that the player has not already shown all their cards
        # but this is not fully necessary as the game will still work (although with a very bad strategy)
        return random.choice(self.get_other_players())

    def ask(self, asktype: str):
        """
        Respond when a player asks to show my lowest/highest card
        """
        if asktype not in ["lowest", "highest"]:
            #raise ValueError("asktype must be either 'lowest' or 'highest'")
            asktypetype = "lowest"  # Default to lowest
        
        index = 0 if asktype == "lowest" else -1
        if self.visible[index]:
            index = 1 if asktype == "lowest" else -2

        print(f"asks {self.name} for their {'next' if index in [1, -2] else ''}{asktype} card")
        print(f"It's a {self.hand[index]}")
        return self.show(index)
    
    def ask_lowest(self):
        return self.ask("lowest")
    
    def ask_highest(self):
        return self.ask("highest")

    def play_full_random(self):
        """
        Play a random action. Possible actions are:
        - ask_lowest(Player)
        - ask_highest(Player)
        - check_table(index)
        """
        results = []
        ask_or_check = random.choices(["ask", "check"], weights=[0.75, 0.25], k=3)
        random_players = random.choices(self.get_other_players(), k=3)
        for i in range(3):
            if ask_or_check[i] == "ask":
                if random.choice(["lowest", "highest"]) == "lowest":
                    res = random_players[i].ask_lowest()
                else:
                    res = random_players[i].ask_highest()
            else:
                print("checks table randomly...")
                res = self.game.table.check_random()
                print(f"Got a {res}")
            
            results.append(res)
            if len(results) == 2 and results[0] != results[1]:
                # We don't have a couple, not allowed to continue
                print("No couple found, turn ends")
                print(results)
                break
            elif len(results) == 3 and results[0] == results[1] == results[2]:
                # We have a trio
                self.game.trio_found(self, results[0])
                break
            else:
                # Continue playing
                pass
        
        self.game.end_turn()

    def play_random(self):
        plychz = random.choices(self.get_other_players(), k=3)
        low_or_high = random.choice(["lowest", "highest"])
        
        results = []
        for i in range(3):
            if ask_or_check[i] == "ask":
                results.append(self.ask(plychz[i], low_or_high))
            else:
                res = self.game.table.check_random()
                if res:
                    results.append(res)
                else:
                    # All cards on the table have been shown
                    results.append(self.ask(plychz[i], low_or_high))
            







            if random.choice(["lowest", "highest"]) == "lowest":
                res1 = self.ask_lowest(plychz[0])
                res2 = self.ask_lowest(plychz[1])
                if res1 == res2:
                    res3 = self.ask_lowest(plychz[2])
                    if res1 == res3:
                        # trio found
                        pass
            else:
                res1 = self.ask_highest(plychz[0])
                res2 = self.ask_highest(plychz[1])
                if res1 == res2:
                    res3 = self.ask_highest(plychz[2])
                    if res1 == res3:
                        # trio found
                        pass
    
        else:
            self.check_table(random.randint(0, len(self.game.table)-1))


class Table(Hand):
    def __init__(self, cards: List[int], game: Game):
        self.game = game
        super().__init__(cards)

    def __str__(self):
        return "Table :\n" + str(self.cards)

    def check(self, index):
        """
        Check a card on the table
        """
        print(f"checks table at position {index}")
        return self.show(index)
    
    def check_random(self):
        """
        Check a random card on the table
        TODO: Check because this random function has memory: does not check already shown cards
        Instead, a turn-scoped memory should be implemented, where function does not check in visible cards
        """
        if self.all_shown():
            return None
        
        # Check that the card is not already shown
        index = random.randint(0, len(self.cards)-1)
        while self.shown[index]:  # Any card visible now is already shown
            index = random.randint(0, len(self.cards)-1)
        
        # Show card
        card = self.check(random.randint(0, len(self.cards)-1))
        return card


In [128]:
trio_game = Game(3)

Distributing cards...
Player 1 discarded trio 12!


In [163]:
for p in trio_game.players[0].get_other_players():
    print("Turn of " + p.name)
    p.play_full_random()

Turn of Player 1
asks Player 0 for their lowest card
It's a 2
asks Player 0 for their highest card
It's a 10
No couple found, turn ends
[2, 10]
Turn of Player 2
asks Player 0 for their lowest card
It's a 2
asks Player 0 for their highest card
It's a 10
No couple found, turn ends
[2, 10]


In [164]:
me = trio_game.players[0]
p1 = trio_game.players[1]
p2 = trio_game.players[2]

# New turn
trio_game.end_turn()
print(me)
print(trio_game.knowledge())


Player 0:
[2, 3, 5, 6, 8, 10]
Player 0:
[2, 3, '?', '?', '?', '?']
Player 1:
[7, 10]
Player 2:
[2, 3, '?', '?', '?', '?', 9, 9]
Table:
[8, 6, 4, 4, 3, 9, 7, 10, '?', 6, 2]
None


In [165]:
print("My turn")
trio_game.table.check(7)
me.ask_highest()
p1.ask_highest()
trio_game.trio_found(me, 10)

My turn
checks table at position 7
asks Player 0 for their highest card
It's a 10
asks Player 1 for their highest card
It's a 10
Player Player 0 found a trio of 10!
Player Player 0 has reached 3 trios and won the game!


In [149]:
trio_game.table.check_random()

checks table at position 10


2

In [145]:
trio_game.trio_found(me, 11)
trio_game.end_turn()

Player Player 0 found a trio of 11!


In [161]:
trio_game.end_turn()

In [132]:
trio_game.print_game()

Table :
[8, 6, 4, 4, 3, 9, 1, 7, 10, 4, 6, 2]
Player 0:
[1, 2, 3, 5, 6, 8, 10, 11]
Player 1:
[1, 7, 10, 11, 11]
Player 2:
[2, 3, 5, 5, 7, 8, 9, 9]
